In [76]:
#Importing OpenCV Library for basic image processing functions
import cv2
# Numpy for array related functions
import numpy as np
# Dlib for deep learning based Modules and face landmark detection
import dlib
#face_utils for basic operations of conversACion
from imutils import face_utils
# Pygame for multimedia applications like sound and graphics
import pygame

In [77]:
#Initializing the camera and taking the instance
cap = cv2.VideoCapture(0)

In [78]:
# Path to the pre-trained shape predictor model
shape_predictor_path = "shape_predictor_81_face_landmarks.dat"

# Initializing the face detector
detector = dlib.get_frontal_face_detector()

# Initializing the landmark predictor
try:
    predictor = dlib.shape_predictor(shape_predictor_path)
    print("Face landmark predictor loaded successfully.")
except RuntimeError as e:
    print("Error:", e)


Face landmark predictor loaded successfully.


In [79]:
#status marking for current state
sleep = 0
drowsy = 0
active = 0
status=""
color=(0,0,0)

In [80]:
def compute(ptA,ptB):
	dist = np.linalg.norm(ptA - ptB)
	return dist

In [81]:
def blinked(a,b,c,d,e,f):
	up = compute(b,d) + compute(c,e)
	down = compute(a,f)
	ratio = up/(2.0*down)

	#Checking if it is blinked
	if(ratio>0.25):
		return 2
	elif(ratio>0.21 and ratio<=0.25):
		return 1
	else:
		return 0

In [82]:
def yawned(a, b, c, d, e, f, g, h):
    # Calculate the distances between relevant landmarks
    upper_lip_dist = compute(a, b) + compute(c, d)
    lower_lip_dist = compute(e, f) + compute(g, h)
    mouth_width = compute(a, c)
    
    # Calculate the ratio between upper and lower lip distances
    ratio = lower_lip_dist / upper_lip_dist
    
    # Check if the ratio exceeds a certain threshold indicating a yawn
    if ratio > 1.5 and mouth_width > 50:  # You might need to adjust these thresholds based on testing
        return True
    else:
        return False

In [83]:
# Initialize pygame
pygame.init()

# Load your music file
music_file_path = "music.wav"  # your actual music file path
pygame.mixer.music.load(music_file_path)

# Function to play music
def play_music():
    pygame.mixer.music.play(-1)  # -1 loops the music indefinitely

# Function to stop music
def stop_music():
    pygame.mixer.music.stop()


In [84]:
# Inside the while loop
while True:
    ret, frame = cap.read()  # Capture a frame
    if not ret:  # Check if the frame was captured successfully
        break  # If not, break out of the loop

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    # detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # The numbers are actually the landmarks which will show eye
        left_blink = blinked(landmarks[36], landmarks[37],
                              landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43],
                               landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        # Check yawning
        yawning_detected = yawned(landmarks[50], landmarks[51], landmarks[52], landmarks[57], 
                                  landmarks[58], landmarks[59], landmarks[61], landmarks[62])

        if yawning_detected > 0.5:
            status = "YAWNING!"
            color = (0, 0, 255)
            play_music()  # Start playing music when yawning
            
        # Now judge what to do for the eye blinks
        
        elif (left_blink == 0 or right_blink == 0):
            sleep += 1
            drowsy = 0
            active = 0
            if (sleep > 6):
                status = "SLEEPING !!!"
                color = (255, 0, 0)
                play_music()
                
        elif (left_blink == 1 or right_blink == 1):
            sleep = 0
            active = 0
            drowsy += 1
            if (drowsy > 6):
                status = "DROWSY !"
                color = (0, 0, 255)
                play_music()  # Start playing music when drowsy

        else:
            drowsy = 0
            sleep = 0
            active += 1
            if (active > 6):
                status = "ACTIVE"
                color = (0, 255, 0)
                stop_music()  # Stop music when active

        cv2.putText(frame, status, (100, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)

    # Check for any key press
    key = cv2.waitKey(1)
    if key == 27 or key != -1: # If any key is pressed
        break ## Break out of the loop

cap.release()  # Release the camera
cv2.destroyAllWindows()  # Close all OpenCV windows